# CMD Windows
ping
- Used to check an IP address by sending packets of data
ipconfig
- Displays all network configuration information
netstat -ano
- Used to check all open ports on the host
tracert "domain"
- Trace routes a domain to its address

# Wireshark
Check chipset of NIC to make sure it allows for monitor/promiscuous mode if you want to capture traffic
Download GeoIP dictionaries to add country/IP owner/Coordinate information to packets and filter on those
Make sure to run in admin or sudo for network access
Use tshark to capture traffic in headless mode
- `tshark -i all` (captures all traffic on all interfaces, but can specify other interface types)
- Best practice is to write to tmp
- Use with other commands to pipe capture to mail or other things, but make sure to control space/stop conditions for file size
**Capture Filters**
- Filter by single host: `ip host == 192.168.0.3`
- Filter by subnet: `ip.addr == 10.0.1.1/24`
- Filter by port: `tcp.port == 80`
- Filter by destination IP: `dst net 192.168.0.0/24`
- Filter by source IP: `src net 192.168`
- Filter by blacklisting ports: `port not 53 and not arp`
**Protocol Authentication Capturing**
- FTP
    - Find plaintext authentication and login information: `ftp contains Password`
- HTTP
    - Find plaintext authentication and login information: `http.authbasic`
- SSH
- Telnet

# Cisco Packet Tracer
### Cisco Router Configuration: CLI Commands
**For more Packet Tracer Switch/Router configs, check ch12 - Cisco Switches and Routers**
```
enable
```
- Allows you to edit router settings (can tell it worked by changing to the # sign)
```
config t
```
- Allows you to enter configure terminal
```
hostname NAME
```
- Change the name of the router/thing
```
no ip domain-lookup
```
- Disable ip lookup "Translating command to domain server" error
```
banner motd #MESSAGE#
```
- Allows you to create a banner message (usually a warning)
```
enable secret PASSWORD
```
- Creates password
```
service password-encryption
```
- Enables passwords to be encrypted instead of being saved as plain text
```
line console 0
```
- Console connection port 0 (lets you access commands for this port)
    ```
    password PASSWORD
    ```
    - Creates a password for accessing the console
```
line vty 0 4
```
- Opens telnet lines 0-4 to be configured and used for incoming traffic
    ```
    logging synchronous
    ```
    - Makes output wait for typing to finish so that it doesn't interfere with commands being written
    ```
    password PASSWORD
    ```
    - Sets password for lines
    ```
    login
    ```
    - Toggles the password login flag on or off for the telnet line
```
int f0/0
```
- Goes into the interface you want to configure (f0/0 in this example)
    ```
    ip address IP SUBNETMASK
    ```
    - Sets this interface to the IP and SUBNETMASK specified
    ```
    no shut
    ```
    - Turns the interface on
```
int se2/0
```
- Goes into the serial connection interface (se2/0 in this example)
    ```
    ip address IP SUBNETMASK
    ```
    - Sets the interface to use the IP and SUBNETMASK specified
    ```
    no shut
    ```
    - Turns the interface on
```
copy run start
```
- Save all configurations **IMPORTANT** (must exit out to base hostname)

### Cisco Router IP Table Creation
```
config t
```
- Get into the configuration controls
```
ip route NETWORKIP SUBNETMASK SERIALCONNECTIONIP
```
- Sets up the ip route with the details
```
ip route 0.0.0.0 0.0.0.0 se2/0
```
- Default Route will set all ip routes not known into the outside network to try to find the correct ip route
```
show ip route
```
- Shows all ip routes on a network

### Cisco Router Access List Configuration
```
access-list **NAME** permit host 10.0.0.1
```
- Creates an access list with NAME and permits a host on that list
```
access-class **NAME** in
```
- Activates an access-list with NAME
```
show access-lists
```
- Shows the access-list configs on the router, must have exited to main

# Nmap
- Read nmap.org to find good documentation
- Make sure to run with permissions (sudo) for stealth scans
**Things to track**
```
1. Check for live systems
2. Check for open ports
3. Scan beyond IDS
4. Perform banner grabbing
5. Scan for vulnerabilities
6. Draw network diagrams
7. Prepare proxies
```
**Scan Types**
- TCP connect() Scan `-sT`
    - Easy to detect
    - Checks for the syn,ack response
    - Name from the connect socket system to create a connection
    - Ports reported as "open" can definitely be connected to
    - An IDS will usually log connection requests to ports not open
- SYN Stealth Scan `-sS` (needs privelages)
    - Half-open scan, connection never fully established
    - Developed with above limitation intended
    - SYN Stealth scan is more stealthy, but still detectable
    - Leverages 3-way handshake protocol specifics
- Other Scans: `-sF, -sN, -sX`
    1. FIN Scan `-sF` - Send packets wiht FIN flag
    2. Null Scan `-sN` - Send packets with no flag set
    3. Xmas Tree Scan `-sX` - Send packets with FIN, PUSH, and URG flags set
- Ping Scan `-sP, -P0` (recommended not to use because it's notorious for being detected and blocked)
    - Ping scan used to see which hosts are online and responding to requests
    - Sends an ICMP echo request first andd if no ICMP echo response i received, it will try a TCP ping which sends a SYN or ACK to any port to determine if it gets a response back.
    - Use the -P0 flag to disable the ICMP echo request and only try a TCP ping
- Version Detection Scan `-sV`
    - Scans services running and tries to find information about those services
    - 'V' flag can be added to any scan
- Agressive Scan `-sA`
    - Does a traceroute, os detection, and service detection
- List Scan `-sL` (passive scan, so totally legal)
    - Does a reverse DNS lookup to view all other DNS names running on the same subnet
- UDP Scan `-sU`
    - UDP scan will check for UDP specific services like DNS and DHCP
- Idle Scan `-sI` (stealthiest of scans)
    - Makes use of a zombie host to never show origin host and can be used for MitM attacks
    - Useful for determining the relationship between two machines
**Useful Flags**
- Use all types `-A`
    - This flag will do a very aggressive flag with a full scan
- Scan all ports `-p-`
    - Scan all 65535 ports
- TCP Ping scan `-PN`
    - Pings hosts which are likely blocking ICMP with TCP instead
- ACK Ping scan `-PA`
    - Pings with both ICMP and TCP (in case ICMP is blocked)
- Decoy scan `-D`
    - Specify to use random IP's `sudo nmap -sV -v 10.0.1.69 -D RND:25`
    - Specify a range of IP's `-D 10.0.1.1, 10.0.1.254`
- Fragmentation `-f`
    - Use with main TCP scan types `-sS, -sF, -sN, -sX` to fragment packets making it harder for a firewall to determine the contents of the packet
- Source port spoof `--source-port`
    - Spoof the source port to take advantage of firewall rules
- MAC address spoof `--spoof-mac`
    - Spoof the source MAC to take advantage of firewall rules
**Scan Timings**
- Slow scan to reduce detection `-T`
    - Use timing `-T 0` (paranoid) or `-T 1` (sneaky) to be avoid detection
        - Paranoid is 5 minutes between packets sent and sneaky is 15 seconds
- Specify timing between probes `--scan-delay`
- Run with threading `-min-parallelism, -max-parallelism`
    - Ex. specifiy a certain number of threads to spawn `nmap -sV 10.0.1.21 -max-parallelism 20`
- Fast scan `-F`
    - Add this flag to substantially increase scan times, and decrease stealth
    
**Scripts**
- see https://nmap.org/nsedoc/ for scripts and categories for nmap
- Kali has most scripts pre-installed, check /usr/share/nmap/scripts
- To use a script `nmap --script scriptname target`